In [6]:
import torch
import segmentation_models_pytorch as smp
from onnx_tf.backend import prepare
import onnx
import tensorflow as tf


Torch to onnx

In [ ]:
def torch2onnx(pt_model_path, onnx_model_path, img_size,model):
    img_size = (img_size, img_size)
    batch_size = 1
    
    model.load_state_dict(torch.load(pt_model_path))
    sample_input = torch.rand((batch_size, 3, *img_size))

    torch.onnx.export(
        model,
        sample_input, 
        onnx_model_path,
        verbose=False,
        input_names=['input'],
        output_names=['output'],
        opset_version=12
    )
torch2onnx('../../model/segmentation/DeepLabV3Plus_callback.pt', '../../model/segmentation/onnx/DeepLabV3Plus.onnx', 224,smp.DeepLabV3Plus('mobilenet_v2', in_channels=3, classes=4))
torch2onnx('../../model/segmentation/NestedUNet_callback.pt', '../../model/segmentation/onnx/NestedUNet.onnx',224 ,smp.UnetPlusPlus('mobilenet_v2', in_channels=3, classes=4))
torch2onnx('../../model/segmentation/PAN_callback.pt', '../../model/segmentation/onnx/PAN.onnx', 224,smp.PAN('mobilenet_v2', in_channels=3, classes=4))
torch2onnx('../../model/segmentation/UNet_callback.pt', '../../model/segmentation/onnx/Unet.onnx', 224,smp.Unet('mobilenet_v2', in_channels=3, classes=4))


Onnx to TF

In [3]:
def onnx2tf(onnx_model_path, tf_model_path):
    onnx_model = onnx.load(onnx_model_path)
    tf_rep = prepare(onnx_model)
    tf_rep.export_graph(tf_model_path)


onnx2tf('../../model/segmentation/onnx/DeepLabV3Plus.onnx', '../../model/segmentation/tf/DeepLabV3Plus')
onnx2tf('../../model/segmentation/onnx/NestedUNet.onnx', '../../model/segmentation/tf/NestedUNet')
onnx2tf('../../model/segmentation/onnx/PAN.onnx', '../../model/segmentation/tf/PAN')
onnx2tf('../../model/segmentation/onnx/Unet.onnx', '../../model/segmentation/tf/Unet')

INFO:tensorflow:Assets written to: ../../model/segmentation/tf/DeepLabV3Plus/assets


INFO:tensorflow:Assets written to: ../../model/segmentation/tf/DeepLabV3Plus/assets


INFO:tensorflow:Assets written to: ../../model/segmentation/tf/NestedUNet/assets


INFO:tensorflow:Assets written to: ../../model/segmentation/tf/NestedUNet/assets


INFO:tensorflow:Assets written to: ../../model/segmentation/tf/PAN/assets


INFO:tensorflow:Assets written to: ../../model/segmentation/tf/PAN/assets


INFO:tensorflow:Assets written to: ../../model/segmentation/tf/Unet/assets


INFO:tensorflow:Assets written to: ../../model/segmentation/tf/Unet/assets


TF to TFlite

In [7]:

def tf2tflite(saved_model_dir, tflite_model_path):
    # Convert the model
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    tflite_model = converter.convert()

    # Save the model
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
        
tf2tflite('../../model/segmentation/tf/DeepLabV3Plus', '../../model/segmentation/tflite/DeepLabV3Plus.tflite')
tf2tflite('../../model/segmentation/tf/NestedUNet', '../../model/segmentation/tflite/NestedUNet.tflite')
tf2tflite('../../model/segmentation/tf/PAN', '../../model/segmentation/tflite/PAN.tflite')
tf2tflite('../../model/segmentation/tf/Unet', '../../model/segmentation/tflite/Unet.tflite')

2024-02-26 10:36:54.984731: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-02-26 10:36:54.984805: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-02-26 10:36:54.987401: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ../../model/segmentation/tf/DeepLabV3Plus
2024-02-26 10:36:55.146977: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-26 10:36:55.147046: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ../../model/segmentation/tf/DeepLabV3Plus
2024-02-26 10:36:55.164727: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2024-02-26 10:36:55.166835: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-26 10:36:55.224802: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on S